In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [2]:
url_data ="https://admin.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E{}&cat=FoulsReceived&agg=PerGame"
years = list(range(2001, 2022))

In [3]:
for year in years:
    with open("fouls_drawn_htmls/{}.html".format(year), "w+" , encoding="utf-8") as file :
        url = url_data.format(year)
        data = requests.get(url)
        file.write(data.text)

In [4]:
dfs = []
for year in years:
    with open("fouls_drawn_htmls/{}.html".format(year)) as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="StatsGridResults table responsive fixed-cols-2 table-left-cols-2 table-expand table-striped table-hover table-noborder table-centered table-condensed")
    player_table = soup.find_all(id="ctl00_ctl00_ctl00_ctl00_maincontainer_maincontent_contentpane_ctl01_ctl01_gvResults")[0]
    fouls_drawn = pd.read_html(str(player_table))[0]
    fouls_drawn["Year"] = year
    dfs.append(fouls_drawn)

In [5]:
fouls_drawn = pd.concat(dfs)
fouls_drawn

,Rank,Player,Team,GamesG,Fouls DrawnFL+,Ave.,Per 40 min.40m,Year
0,1,"HUGGINS, ROGERHUGGINS, R.",CHASpirou BasketSpirou Basket,14,100,7.14,9.58,2001
1,2,"SKELIN, MATESKELIN, M.",MESKRKA Novo MestoKRKA Novo Mesto,14,99,7.07,8.64,2001
2,3,"FUCKA, GREGORFUCKA, G.",FORSkipper BolognaSkipper Bologna,20,128,6.40,8.02,2001
3,4,"BODIROGA, DEJANBODIROGA, D.",PAOPanathinaikos AthensPanathinaikos Athens,22,139,6.32,7.83,2001
4,5,"LAKOVIC, JAKALAKOVIC, J.",MESKRKA Novo MestoKRKA Novo Mesto,14,88,6.29,7.74,2001
...,...,...,...,...,...,...,...,...
27,28,"WILBEKIN, SCOTTIEWILBEKIN, S.",MTAMaccabi Playtika Tel AvivMaccabi Playtika T...,35,108,3.09,4.25,2021
28,29,"HEZONJA, MARIOHEZONJA, M.",UNKUNICS KazanUNICS Kazan,25,76,3.04,4.10,2021
29,30,"JONES, CHRISJONES, C.",ASVLDLC ASVEL VilleurbanneLDLC ASVEL Villeurbanne,31,94,3.03,4.41,2021
30,31,"MACON JR, DARYLMACON, D.",PAOPanathinaikos OPAP AthensPanathinaikos OPAP...,31,93,3.00,4.64,2021


In [6]:
engine = create_engine('sqlite:///fouls_drwan_leaders.db')
Session = sessionmaker(bind=engine)
session = Session()

In [7]:
fouls_drawn.to_sql(
    name = "fouls_drawn_leaders",
    con = engine,
    index = True,
    if_exists = "replace"
)

813

In [ ]:
fouls_drawn_leaders_select = session.execute('select * from "fouls_drawn_leaders"')
fouls_drawn_leaders_select.all()